# 나이브 베이즈 분류기(Naive Bayes Classifier)

1. 베이즈의 정리 ( 조건부 확률)
- $P(A) 가 A가 일어날 확률, P(B)가 B가 일어날 확률, P(B|A)가 A가 일어나고나서 B가 일어날 확률$
$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$


- 입력 텍스트(메일의 본문)이 주어졌을 때, 입력 텍스트가 정상 메일인지 스팸 메일인지 구분하기 위한 확률을 이와 같이 표현할 수 있습니다.
    - P(정상 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 정상 메일일 확률
        - P(정상 메일 | 입력 텍스트) = (P(입력 텍스트 | 정상 메일) × P(정상 메일)) / P(입력 텍스트)
    - P(스팸 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 스팸 메일일 확률
        - P(스팸 메일 | 입력 텍스트) = (P(입력 텍스트 | 스팸 메일) × P(스팸 메일)) / P(입력 텍스트)

![](2022-03-03-13-00-50.png)

위와 같은 훈련 데이터가 있을 때,

"you free lottery" 라는 입력 테스트는 정상 메일일 확률과 스팸 메일일 확률을 구한다.

$$P(정상 메일 | 입력 텍스트) = P(you | 정상 메일) × P(free | 정상 메일) × P(lottery | 정상 메일) × P(정상 메일)$$
$$P(스팸 메일 | 입력 텍스트) = P(you | 스팸 메일) × P(free | 스팸 메일) × P(lottery | 스팸 메일) × P(스팸 메일)$$

$$P(정상 메일) = P(스팸 메일) = 총 메일 6개 중 3개 = 0.5$$

- P(you|정상 메일) 구하기
    - 정상 메일에 등장한 모든 단어의 빈도 수의 총합을 분모,, 정상 메일에서 you가 총 등장한 빈도의 수를 분자로 하는 것
    - 2/10

- 따라서 아래의 식에 따라 위 "you free lottery" 는 스팸으로 분류된다.
    - P(정상 메일 | 입력 텍스트) = 2/10 × 2/10 × 0/10 = 0
    - P(스팸 메일 | 입력 텍스트) = 2/10 × 3/10 × 2/10 = 0.012

- 하지만, 훈련 텍스트에 없다고 0 이 되는 단점이 있다.
    - 이를 방지하기 위해 나이브 베이즈 분류기에서는 각 단어에 대한 확률의 분모,분자에 ㅈ숫자를 더해 분자가 0 이 되는 것을 방지하는 라플라스 스무딩을 사용하기도 한다!


### 3. 뉴스그룹 데이터 분류하기
- Twenty Newsgroups
    - 20 개의 다른 주제를 가진 18,846개의 뉴스그룹 데이터 제공

In [24]:
from sklearn.datasets import fetch_20newsgroups
newsdata = fetch_20newsgroups(subset='train')
    # subset 에 'all'을 넣으면 전체 데이터 다운
    # 'train' 넣으면 훈련 데이터
    # 'test' 넣으면 테스트 데이터 다운
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [25]:
print(len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))
# 훈련용 샘플은 11314 개의 샘플과 20개의 카테고리를 갖고 있다.

11314 11314 20 11314


In [26]:
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [27]:
print("카테고리 레이블",newsdata.target[0])
print("메일 내용")
print(newsdata.data[0])

카테고리 레이블 7
메일 내용
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







### 2. 나이브 베이즈 분류
- 데이터 토큰화 필요(BoW)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score # 정확도 계산

dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


- 자동으로 DTM 완료. 
    - 11,314 훈련용 샘플 (DTM 관점에서는 문서의 수)
    - 130.107은 전체 훈련 데이터에 등장한 단어의 수
- DTM 행렬 대신 TF-IDF 가중치를 적용한 행렬로 성능 개선 가능
    - *주의* 항상 뛰어나지는 않음

In [29]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


In [30]:
# 나이브 베이즈 모델 사용
# TF-IDF 행렬과 11,314 훈련 데이터에 대한 레이블이 들어감
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)

MultinomialNB()

In [31]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=None)
# alpha 1.0 은 라플라스 스무딩이 적용됨

MultinomialNB(fit_prior=None)

In [32]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True)

# test data 에는 fit_transform 이 아님!!!!!
# 왜냐하면 학습에 가중치를 그대로 가져와서 사용해야 하기 때문
X_test_dtm = dtmvector.transform(newsdata_test.data)
tfidfv_test = tfidf_transformer.transform(X_test_dtm)

predicted = mod.predict(tfidfv_test)
print("정확도 : ", accuracy_score(newsdata_test.target, predicted))

(7532, 130107)
정확도 :  0.7738980350504514
